In [3]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [5]:
def sim(n=2, T=1000, alpha = 0.9, beta = 0.2, gamma = 0.1, lamb0=1, lamb1=5):
    #Defining the three possible values of C
    ck = np.arange(3)   
    
    #Gamma matrix for assigning P(C_t | C_{t-1})
    Gamma = np.array([[1-gamma, 0, gamma], 
                      [0, 1-gamma, gamma], 
                      [beta/2, beta/2, 1-beta]])
    
    #Creating random variables with probabilities based of the gamma matrix
    C_transition = [
        stats.rv_discrete(values=(ck,Gamma[0,])),#P(C_t |C_{t-1} = 0)
        stats.rv_discrete(values=(ck,Gamma[1,])),#P(C_t |C_{t-1} = 1)
        stats.rv_discrete(values=(ck,Gamma[2,])),#P(C_t |C_{t-1} = 2)
    ]

    #Creating output vector of C's
    C = np.zeros(T, np.int64)
    #Initializing the C vector
    C[0] = 2

    #Filling up the C-vector with values
    for i in range(T-1):
        C[i+1] = C_transition[C[i]].rvs()
    
    #CPT of Z
    Z_given_C = np.array([1-alpha, alpha, 0.5])  #P(Z = 1| C =c)
    
    #Initializing Z. size=[n,T] specifies we need to create n copies of a series of T simulations
    Z = stats.bernoulli(Z_given_C[C]).rvs(size=[n,T])
    #input:
    '''
    [C_1, C_2, ... C_T]
    '''
    #output: 
    '''[   
        [Z_11, Z_21, Z_31, ... , Z_T1]
        [Z_12, Z_22, Z_32, ... , Z_T2]
        ...
        [Z_1n, Z_2n, Z_3n, ... , Z_Tn]
    ]'''

    #Initialize X   
    X = stats.poisson(np.where(Z, lamb1, lamb0)).rvs()
    #input
    '''[   
        [f(Z_11), f(Z_21), f(Z_31), ... , f(Z_T1)]
        [f(Z_12), f(Z_22), f(Z_32), ... , f(Z_T2)]
        ...
        [f(Z_1n), f(Z_2n), f(Z_3n), ... , f(Z_Tn)]
        ]    
        where
        f(z) = lamb0+(lamb1-lamb0)*z
    '''

    #output
    '''[   
        [X_11, X_21, X_31, ... , X_T1]
        [X_12, X_22, X_32, ... , X_T2]
        ...
        [X_1n, X_2n, X_3n, ... , X_Tn]
    ]'''

    return C,Z,X

C, Z, X = sim(n = 2,T=10)

In [ ]:
# inference of z, returns unnormalized probabilities
def z_inf(x, mu, sigma, p):
    M, n = x.shape
    K = mu.shape[0]
    c = np.sqrt(2 * np.pi) ** n
    f = np.zeros(shape = (M, K))
    for k in range(K):
        exponential = np.exp(np.sum(- ((x - mu[k, :]) ** 2) / (2 * (sigma[k, :] ** 2)), axis = 1))
        f[:, k] = p[k] * exponential / (c * np.prod(sigma[k, :]))
    return(f)

# This function assumes a matrix of (not necessarily normalized) probabilities of z
def MAP(z):
    return((z >= np.max(z, axis = 1, keepdims = True)).astype(int))

In [6]:
def learn_par(C,Z,X, Print = False):
    n = X.shape[0]
    T = X.shape[1]

    # Estimating lambda's
    lambda1 = sum(sum((Z[:,t] * X[:,t] for t in range(T)))) / (sum(sum(Z)))
    lambda0 = sum(sum(((np.ones(shape = n)-Z[:,t]) * X[:,t] for t in range(T)))) / (n*T - sum(sum(Z)))

    # Indicator functions of C 
    C_0 = [C[t] == 0 for t in range(T)]
    C_1 = [C[t] == 1 for t in range(T)]
    C_2 = [C[t] == 2 for t in range(T)]

    # Estimating alpha_hat
    if (sum(C_1) > 0):
        sum1 = sum(sum(Z[i,t] * C_1[t] for i in range(n)) for t in range(T)) / (2 * sum(C_1) * n)
    else:
        sum1 = 0
    if (sum(C_0) > 0):
        sum2 = sum(sum((1-Z[i,t]) * C_0[t] for i in range(n)) for t in range(T)) / (2 * sum(C_0) * n)
    else:
        sum2 = 0
    alpha_hat = sum1 + sum2

    # Estimating beta hat
    beta_hat = sum((C_0[t+1] + C_1[t+1])*C_2[t] for t in range(T-1)) / (sum(C_2[0:T-1]))

    # Estimating gamma hat
    gamma_hat = sum((C_2[t+1])*(C_1[t]+C_0[t]) for t in range(T-1)) / (sum(C_1[0:T-1]+ C_0[0:T-1]))

    if (Print == True):
        print("lambda0_hat is: ", lambda0, "\nlambda1_hat is: ", lambda1, "\nalpha_hat is:", alpha_hat, "\nbeta_hat is: ", beta_hat, "\ngamme_hat is: ", gamma_hat)
    
    return lambda0 ,lambda1, alpha_hat, beta_hat, gamma_hat

In [ ]:
def EM(C, Z, X, alpha = 0.9, beta = 0.2, gamma = 0.1, lamb0=1, lamb1=5, N = 10):

    for i in range(N):

        # Inference - infer the most probable C,Z,X values

        C_most_prob = MAP(C_inf(X, par_est))
        Z_most_prob = MAP(Z_inf(X, par_est))

        # Learning - computing the most probable parameter estimates

        par_est = learn_par(C_most_prob, Z_most_prob, X, Print = False)    

    return par_est